In [ ]:
%matplotlib inline

In [ ]:
import datetime

import requests
import pandas as pd

from bs4 import BeautifulSoup

In [ ]:
def pegar_links():
    lista_links = []
    for pagina in range(0, 1000, 100):
        url = "http://www2.planalto.gov.br/acompanhe-o-planalto/"\
              "discursos/discursos-da-presidenta/?b_start:int={0}".format(pagina)
        
        r = requests.get(url)
        content = BeautifulSoup(r.content, "lxml")
        links = content.find_all("a", {'class': 'summary url'})
        links = [l['href'] for l in links]
        
        lista_links.extend(links)
        
    return lista_links

In [ ]:
lista_links = pegar_links()

In [ ]:
def baixar_discurso(url):
    try:
        r = requests.get(url)

        content = BeautifulSoup(r.content, "lxml")

        titulo = content.find("h1", {"class": "documentFirstHeading"}).text
        description = content.find("div", {"class": "description"})

        if description is not None:
            description = description.text
        
        document_published = (
            content.find("span", {"class": "documentPublished"}).text.strip()
            .replace("publicado", "").strip().replace(",", "")
        )
        document_published = datetime.datetime.strptime(document_published, "%d/%m/%Y %Hh%M")

        document_modified = content.find("span", {"class": "documentModified"})
        
        if document_modified is not None:
            document_modified = (
                document_modified.text.strip().replace(u"última modificação", "")
                .strip().replace(",", "")
            )

            document_modified = datetime.datetime.strptime(document_modified, "%d/%m/%Y %Hh%M")

        texto = content.find("div", {"id": "parent-fieldname-text"}).text

        tags = content.find("div", {"id": "category"})
        if tags is not None:
            tags = [t.text for t in tags.find_all("a")]
        else:
            tags = []

        assuntos = content.find("div", {"id": "category-vcge"})
        if assuntos is not None:
            assuntos = [a.text for a in assuntos.find_all("a")]
        else:
            assuntos = []
        
        d = {
            'titulo': titulo,
            'descricao': description,
            'data_publicacao': document_published,
            'data_modificacao': document_modified,
            'texto': texto,
            'tags': tags,
            'assuntos': assuntos,
            'url': url
        }
        return d
    
    except Exception as e:
        print "Erro ao baixar: {0} - {1}".format(url, e)
        return None

In [ ]:
discursos = []
for url in lista_links:
    discurso = baixar_discurso(url)
    if discurso is not None:
        discursos.append(discurso)